In [199]:
import pandas as pd
import numpy as np


import matplotlib.pyplot as plt
plt.style.use ('ggplot')
import seaborn as sns
%matplotlib inline
sns.set_style(style="whitegrid")

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.decomposition import NMF
from sklearn.feature_extraction import text

from bs4 import BeautifulSoup

import nltk, string, contractions

from gensim.corpora.dictionary import Dictionary
from gensim.models.nmf import Nmf
from gensim.models.coherencemodel import CoherenceModel

from operator import itemgetter

In [204]:
df = pd.read_csv('data/modeling_ready_microwave1')

In [205]:
df.dropna(inplace=True)

In [206]:
df.review_date = pd.to_datetime(df.review_date)

In [207]:
def vectorize_this(max_features, min_df, max_df, ngram_max):
    vectorizer = TfidfVectorizer(tokenizer = lemmatize_text,
                             stop_words= review_stop_words,
                             max_features = max_features,
                             min_df = min_df,
                             max_df =  max_df,
                             ngram_range=(1, ngram_max)
                            )
    X = df['model_ready']
    X = vectorizer.fit_transform(X)
    return X, vectorizer

In [276]:
X, vectorizer = vectorize_this(10000, 2, .95, 2)

In [277]:
def make_nmf(n_components, alpha, X):
    nmf = NMF(
            n_components=n_components,
            init='nndsvd',
            random_state=12345,
            alpha = alpha
            ).fit(X)

    W = nmf.fit_transform(X)
    H = nmf.components_
    return nmf, W, H

In [284]:
nmf, W, H, = make_nmf(22, .1, X)

In [285]:
def topic_keywords(vectorizer=vectorizer, lda_model=nmf, n_words=10):
    keywords = np.array(vectorizer.get_feature_names())
    topic_keywords = []
    for topic_weights in lda_model.components_:
        top_keyword_locs = (-topic_weights).argsort()[:n_words]
        topic_keywords.append(keywords.take(top_keyword_locs))
    return topic_keywords

In [286]:
topic_keywords = topic_keywords()        

In [287]:
topic_keywords

[array(['open', 'button', 'door', 'push', 'door open', 'open door',
        'push button', 'button open', 'hand', 'pull'], dtype='<U23'),
 array(['door', 'latch', 'close', 'door latch', 'problem', 'slam', 'shut',
        'door close', 'broke', 'hard'], dtype='<U23'),
 array(['month', 'service', 'repair', 'warranty', 'customer', 'center',
        'died', 'cost', 'customer service', 'called'], dtype='<U23'),
 array(['large', 'fit', 'dish', 'needed', 'size', 'wanted', 'cabinet',
        'capacity', 'perfect', 'large capacity'], dtype='<U23'),
 array(['sensor', 'reheat', 'sensor reheat', 'function', 'defrost', 'food',
        'reheat function', 'button', 'turbo', 'warm'], dtype='<U23'),
 array(['time', 'cooking', 'cooking time', 'long', 'long time', 'second',
        'minute', 'start', 'arrived time', 'run'], dtype='<U23'),
 array(['power', 'level', 'power level', 'power setting', 'setting', 'set',
        'off', 'lower', 'cooking', 'lower power'], dtype='<U23'),
 array(['working', 'stoppe

In [255]:
def topic_featuring(n_components=10, n_words=10):
    
    nmf, W, H = make_nmf(n_components, .1, X)
    

    # Topic - Keywords Dataframe
    df_topic_keywords = pd.DataFrame(topic_keywords)
    df_topic_keywords.columns = ['Word '+str(i) for i in range(df_topic_keywords.shape[1])]
    df_topic_keywords.index = ['Topic '+str(i) for i in range(df_topic_keywords.shape[0])]

    Topics_theme = range(n_components)
    df_topic_keywords['topic_theme'] = Topics_theme
    df_topic_keywords.set_index('topic_theme', inplace=True)
    return df_topic_keywords.T

In [290]:
topic_featuring(22)

topic_theme,0,1,2,3,4,5,6,7,8,9,...,12,13,14,15,16,17,18,19,20,21
Word 0,open,door,month,large,sensor,time,power,working,lasted,price,...,powerful,need,popcorn,used,fine,big,inverter,heat,item,space
Word 1,button,latch,service,fit,reheat,cooking,level,stopped,month,best,...,powerful large,replaced,setting,look,worked,turkey,technology,food,advertised,counter
Word 2,door,close,repair,dish,sensor reheat,cooking time,power level,stopped working,died,store,...,room,control,bag,little,worked fine,big powerful,inverter technology,evenly,time item,counter space
Word 3,push,door latch,warranty,needed,function,long,power setting,quit working,second,right,...,loud,spacious,button,long used,working fine,big inside,cooking,heat food,size,size
Word 4,door open,problem,customer,size,defrost,long time,setting,quit,going,size,...,big powerful,look,popcorn setting,time used,job,expected,defrosting,quickly,hot,kitchen
Word 5,open door,slam,center,wanted,food,second,set,month,lasted 10,value,...,hot,panel,pop,getting,control,inside,defrost,fast,better,bit
Word 6,push button,shut,died,cabinet,reheat function,minute,off,month working,performance,delivery,...,spacious powerful,new,popcorn button,daily,cabinet worked,job,meat,food evenly,cooking,huge
Word 7,button open,door close,cost,capacity,button,start,lower,given,10,best price,...,simple,control panel,burn,think,broke,big big,quite,heat evenly,large item,little
Word 8,hand,broke,customer service,perfect,turbo,arrived time,cooking,color,owned,needed,...,spacious,need look,size,getting used,office,big need,difference,food quickly,issue,room
Word 9,pull,hard,called,large capacity,warm,run,lower power,completely stopped,lasted month,price quality,...,machine,size,make,used daily,month,room,weight,heating,food item,little bit


In [257]:
def nmf_featurizer(max_features=10000, min_df=4, max_df=.8, ngram_max = 2, n_components=10, n_words=10, column_names_known = 'n'):
    X, vectorizer = vectorize_this(max_features, min_df, max_df, ngram_max)
    
    nmf, W, H = make_nmf(n_components, .1, X)
    
    # Topic - Keywords Dataframe
    df_topic_keywords = pd.DataFrame(topic_keywords)
    df_topic_keywords.columns = ['Word '+str(i) for i in range(df_topic_keywords.shape[1])]
    df_topic_keywords.index = ['Topic '+str(i) for i in range(df_topic_keywords.shape[0])]
    
    if column_names_known == 'n':
        Topics_theme = range(n_components)
    elif column_names_known == 'Y':
        Topics_theme = topic_labels
    df_topic_keywords['topic_theme'] = Topics_theme
    df_topic_keywords.set_index('topic_theme', inplace=True)
    return df_topic_keywords.T

In [291]:
nmf_featurizer(max_df=.95, ngram_max=2, n_components=22)

topic_theme,0,1,2,3,4,5,6,7,8,9,...,12,13,14,15,16,17,18,19,20,21
Word 0,open,door,month,large,sensor,time,power,working,lasted,price,...,powerful,need,popcorn,used,fine,big,inverter,heat,item,space
Word 1,button,latch,service,fit,reheat,cooking,level,stopped,month,best,...,powerful large,replaced,setting,look,worked,turkey,technology,food,advertised,counter
Word 2,door,close,repair,dish,sensor reheat,cooking time,power level,stopped working,died,store,...,room,control,bag,little,worked fine,big powerful,inverter technology,evenly,time item,counter space
Word 3,push,door latch,warranty,needed,function,long,power setting,quit working,second,right,...,loud,spacious,button,long used,working fine,big inside,cooking,heat food,size,size
Word 4,door open,problem,customer,size,defrost,long time,setting,quit,going,size,...,big powerful,look,popcorn setting,time used,job,expected,defrosting,quickly,hot,kitchen
Word 5,open door,slam,center,wanted,food,second,set,month,lasted 10,value,...,hot,panel,pop,getting,control,inside,defrost,fast,better,bit
Word 6,push button,shut,died,cabinet,reheat function,minute,off,month working,performance,delivery,...,spacious powerful,new,popcorn button,daily,cabinet worked,job,meat,food evenly,cooking,huge
Word 7,button open,door close,cost,capacity,button,start,lower,given,10,best price,...,simple,control panel,burn,think,broke,big big,quite,heat evenly,large item,little
Word 8,hand,broke,customer service,perfect,turbo,arrived time,cooking,color,owned,needed,...,spacious,need look,size,getting used,office,big need,difference,food quickly,issue,room
Word 9,pull,hard,called,large capacity,warm,run,lower power,completely stopped,lasted month,price quality,...,machine,size,make,used daily,month,room,weight,heating,food item,little bit


In [292]:
vocabulary = np.array(vectorizer.get_feature_names())

In [ ]:
def label_topics(H, vocabulary):
    '''
    Print the most influential words of each latent topic, and prompt the user
    to label each topic. The user should use their humanness to figure out what
    each latent topic is capturing.
    '''
    topic_labels = []
    for i, row in enumerate(H):
        top_five = np.argsort(row)[::-1][:12]
        print('topic', i)
        print('-->', ' '.join(vocabulary[top_five]))
        label = input('please label this topic: ')
        topic_labels.append(label)
        print()
    return topic_labels

In [293]:
nmf_featurizer(max_df=.95, ngram_max=2, n_components=22)

topic_theme,0,1,2,3,4,5,6,7,8,9,...,12,13,14,15,16,17,18,19,20,21
Word 0,open,door,month,large,sensor,time,power,working,lasted,price,...,powerful,need,popcorn,used,fine,big,inverter,heat,item,space
Word 1,button,latch,service,fit,reheat,cooking,level,stopped,month,best,...,powerful large,replaced,setting,look,worked,turkey,technology,food,advertised,counter
Word 2,door,close,repair,dish,sensor reheat,cooking time,power level,stopped working,died,store,...,room,control,bag,little,worked fine,big powerful,inverter technology,evenly,time item,counter space
Word 3,push,door latch,warranty,needed,function,long,power setting,quit working,second,right,...,loud,spacious,button,long used,working fine,big inside,cooking,heat food,size,size
Word 4,door open,problem,customer,size,defrost,long time,setting,quit,going,size,...,big powerful,look,popcorn setting,time used,job,expected,defrosting,quickly,hot,kitchen
Word 5,open door,slam,center,wanted,food,second,set,month,lasted 10,value,...,hot,panel,pop,getting,control,inside,defrost,fast,better,bit
Word 6,push button,shut,died,cabinet,reheat function,minute,off,month working,performance,delivery,...,spacious powerful,new,popcorn button,daily,cabinet worked,job,meat,food evenly,cooking,huge
Word 7,button open,door close,cost,capacity,button,start,lower,given,10,best price,...,simple,control panel,burn,think,broke,big big,quite,heat evenly,large item,little
Word 8,hand,broke,customer service,perfect,turbo,arrived time,cooking,color,owned,needed,...,spacious,need look,size,getting used,office,big need,difference,food quickly,issue,room
Word 9,pull,hard,called,large capacity,warm,run,lower power,completely stopped,lasted month,price quality,...,machine,size,make,used daily,month,room,weight,heating,food item,little bit


In [294]:
topic_labels = label_topics(H,vocabulary)

topic 0
--> open button door push door open open door push button button open hand pull push open open push
please label this topic: open door button

topic 1
--> door latch close door latch problem slam shut door close broke hard slam door turn
please label this topic: door shutting

topic 2
--> month service repair warranty customer center died cost customer service called week 3
please label this topic: customer service and support

topic 3
--> large fit dish needed size wanted cabinet capacity perfect large capacity exactly larger
please label this topic: internal capacity

topic 4
--> sensor reheat sensor reheat function defrost food reheat function button turbo warm hot minute
please label this topic: programmed functions

topic 5
--> time cooking cooking time long long time second minute start arrived time run time used tell
please label this topic: cooking time

topic 6
--> power level power level power setting setting set off lower cooking lower power cycle display
please labe

In [295]:
nmf_featurizer(max_df=.95, ngram_max=2, n_components=22, column_names_known = 'Y')

topic_theme,open door button,door shutting,customer service and support,internal capacity,programmed functions,cooking time,power level settings,short lifespan,lifespan,perceived value,...,power,nothing,popcorn,daily usage,meets expectations,size,inverter,evenly cooks,nothing,external size
Word 0,open,door,month,large,sensor,time,power,working,lasted,price,...,powerful,need,popcorn,used,fine,big,inverter,heat,item,space
Word 1,button,latch,service,fit,reheat,cooking,level,stopped,month,best,...,powerful large,replaced,setting,look,worked,turkey,technology,food,advertised,counter
Word 2,door,close,repair,dish,sensor reheat,cooking time,power level,stopped working,died,store,...,room,control,bag,little,worked fine,big powerful,inverter technology,evenly,time item,counter space
Word 3,push,door latch,warranty,needed,function,long,power setting,quit working,second,right,...,loud,spacious,button,long used,working fine,big inside,cooking,heat food,size,size
Word 4,door open,problem,customer,size,defrost,long time,setting,quit,going,size,...,big powerful,look,popcorn setting,time used,job,expected,defrosting,quickly,hot,kitchen
Word 5,open door,slam,center,wanted,food,second,set,month,lasted 10,value,...,hot,panel,pop,getting,control,inside,defrost,fast,better,bit
Word 6,push button,shut,died,cabinet,reheat function,minute,off,month working,performance,delivery,...,spacious powerful,new,popcorn button,daily,cabinet worked,job,meat,food evenly,cooking,huge
Word 7,button open,door close,cost,capacity,button,start,lower,given,10,best price,...,simple,control panel,burn,think,broke,big big,quite,heat evenly,large item,little
Word 8,hand,broke,customer service,perfect,turbo,arrived time,cooking,color,owned,needed,...,spacious,need look,size,getting used,office,big need,difference,food quickly,issue,room
Word 9,pull,hard,called,large capacity,warm,run,lower power,completely stopped,lasted month,price quality,...,machine,size,make,used daily,month,room,weight,heating,food item,little bit


In [226]:
def softmax(v, temperature=1.0):
    '''
    A heuristic to convert arbitrary positive values into probabilities.
    See: https://en.wikipedia.org/wiki/Softmax_function
    '''
    expv = np.exp(v / temperature)
    s = np.sum(expv)
    return expv / s

In [227]:
def analyze_reviews(W, topic_labels):
    '''
    Print an analysis of a single Amazon review, including the review ID
    and a summary of which topics it represents. The topics are identified
    via the hand-labels which were assigned by the user.
    '''
    topic_count = len(topic_labels)
    topic_percentages = [[] for i in range(0, topic_count)]

    for idx, i in enumerate(W):  
        probs = softmax(W[idx], temperature=0.01)
        topic_list_counter = 0
        for prob, label in zip(probs, topic_labels):
            topic_percentages[topic_list_counter].append(round(prob, 5))
            topic_list_counter += 1
        
    return pd.DataFrame(dict(zip(topic_labels, topic_percentages)))

In [296]:
top_df = analyze_reviews(W, topic_labels)

In [297]:
top_df

,open door button,door shutting,customer service and support,internal capacity,programmed functions,cooking time,power level settings,short lifespan,lifespan,perceived value,...,wattage,power,nothing,popcorn,daily usage,meets expectations,size,inverter,evenly cooks,external size
0,0.00000,0.99991,0.00000,0.00000,0.00000,0.00008,0.00000,0.00000,0.00000,0.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
1,0.00001,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,...,0.00000,0.00005,0.00000,0.00000,0.00000,0.00000,0.00001,0.00000,0.00857,0.99129
2,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
3,0.01010,0.01460,0.70185,0.01297,0.01010,0.01010,0.01010,0.01038,0.01010,0.01323,...,0.02999,0.01166,0.01010,0.01010,0.01010,0.01953,0.01085,0.01010,0.01017,0.01393
4,0.00023,0.99965,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00001,0.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1843,0.00000,0.00000,0.00000,0.00000,0.00000,0.00003,0.00000,0.00000,0.00000,0.00000,...,0.00000,0.99987,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
1844,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.21154,0.00000
1845,0.00000,0.00000,0.00000,0.03576,0.00000,0.00000,0.00000,0.00000,0.00000,0.00001,...,0.00000,0.00000,0.00000,0.58272,0.00000,0.00000,0.00000,0.00000,0.38147,0.00000
1846,0.00001,0.00001,0.00002,0.00001,0.00001,0.99974,0.00001,0.00001,0.00001,0.00001,...,0.00001,0.00001,0.00001,0.00001,0.00001,0.00001,0.00001,0.00001,0.00006,0.00001


In [298]:
top_df.loc[0]

open door button                    0.00000
door shutting                       0.99991
customer service and support        0.00000
internal capacity                   0.00000
programmed functions                0.00000
cooking time                        0.00008
power level settings                0.00000
short lifespan                      0.00000
lifespan                            0.00000
perceived value                     0.00000
replacement (expect consistency)    0.00000
wattage                             0.00000
power                               0.00000
nothing                             0.00000
popcorn                             0.00000
daily usage                         0.00000
meets expectations                  0.00000
size                                0.00000
inverter                            0.00000
evenly cooks                        0.00000
external size                       0.00000
Name: 0, dtype: float64

In [299]:
feature_score = {}
for _ in range(len(top_df.columns)):
    key = top_df.columns[_]
    feature_score[key] = round(sum(top_df[key])/(len(top_df)),5)

In [300]:
feature_score

{'open door button': 0.05206,
 'door shutting': 0.06647,
 'customer service and support': 0.08608,
 'internal capacity': 0.05858,
 'programmed functions': 0.05921,
 'cooking time': 0.04812,
 'power level settings': 0.04878,
 'short lifespan': 0.03611,
 'lifespan': 0.03743,
 'perceived value': 0.04224,
 'replacement (expect consistency)': 0.05941,
 'wattage': 0.03606,
 'power': 0.03102,
 'nothing': 0.03194,
 'popcorn': 0.03837,
 'daily usage': 0.03589,
 'meets expectations': 0.02967,
 'size': 0.03347,
 'inverter': 0.0359,
 'evenly cooks': 0.0529,
 'external size': 0.04508}

In [301]:
sorted(feature_score.items(), key= lambda x: x[1])

[('meets expectations', 0.02967),
 ('power', 0.03102),
 ('nothing', 0.03194),
 ('size', 0.03347),
 ('daily usage', 0.03589),
 ('inverter', 0.0359),
 ('wattage', 0.03606),
 ('short lifespan', 0.03611),
 ('lifespan', 0.03743),
 ('popcorn', 0.03837),
 ('perceived value', 0.04224),
 ('external size', 0.04508),
 ('cooking time', 0.04812),
 ('power level settings', 0.04878),
 ('open door button', 0.05206),
 ('evenly cooks', 0.0529),
 ('internal capacity', 0.05858),
 ('programmed functions', 0.05921),
 ('replacement (expect consistency)', 0.05941),
 ('door shutting', 0.06647),
 ('customer service and support', 0.08608)]

In [302]:
topic_words_df = nmf_featurizer(column_names_known ='Y')
topic_words_df

topic_theme,open door button,door shutting,customer service and support,internal capacity,programmed functions,cooking time,power level settings,short lifespan,lifespan,perceived value,...,power,nothing,popcorn,daily usage,meets expectations,size,inverter,evenly cooks,nothing,external size
Word 0,open,door,month,large,sensor,time,power,working,lasted,price,...,powerful,need,popcorn,used,fine,big,inverter,heat,item,space
Word 1,button,latch,service,fit,reheat,cooking,level,stopped,month,best,...,powerful large,replaced,setting,look,worked,turkey,technology,food,advertised,counter
Word 2,door,close,repair,dish,sensor reheat,cooking time,power level,stopped working,died,store,...,room,control,bag,little,worked fine,big powerful,inverter technology,evenly,time item,counter space
Word 3,push,door latch,warranty,needed,function,long,power setting,quit working,second,right,...,loud,spacious,button,long used,working fine,big inside,cooking,heat food,size,size
Word 4,door open,problem,customer,size,defrost,long time,setting,quit,going,size,...,big powerful,look,popcorn setting,time used,job,expected,defrosting,quickly,hot,kitchen
Word 5,open door,slam,center,wanted,food,second,set,month,lasted 10,value,...,hot,panel,pop,getting,control,inside,defrost,fast,better,bit
Word 6,push button,shut,died,cabinet,reheat function,minute,off,month working,performance,delivery,...,spacious powerful,new,popcorn button,daily,cabinet worked,job,meat,food evenly,cooking,huge
Word 7,button open,door close,cost,capacity,button,start,lower,given,10,best price,...,simple,control panel,burn,think,broke,big big,quite,heat evenly,large item,little
Word 8,hand,broke,customer service,perfect,turbo,arrived time,cooking,color,owned,needed,...,spacious,need look,size,getting used,office,big need,difference,food quickly,issue,room
Word 9,pull,hard,called,large capacity,warm,run,lower power,completely stopped,lasted month,price quality,...,machine,size,make,used daily,month,room,weight,heating,food item,little bit


In [271]:
review_df = pd.concat([df, top_df.reindex(df.index)], axis=1)[['review_body',
                                                               'cooking time', 'door', 
                                                               'customer service & support',
                                                               'external size', 
                                                               'pre-programmed functions',
                                                               'internal size', 'power controls', 
                                                               'short product lifetime',
                                                               'product lifetime', 
                                                               'perceived value', 
                                                               'replacement for older microwave',
                                                               'power and wattage', 'power',
                                                               'cooks evenly', 'popcorn', 
                                                               'regular use']]

In [303]:
print(review_df.review_body[0])
print(review_df.loc[0])

I have had this microwave for just over 3 years and the door won't close, or the microwave doesn't know that the door is closed so it won't turn on. It is rather irritating to be having to keep slamming and pressing and shaking the door to make it start. Many a times we end up heating/cooking on the stove. I do not think it is worth the over $100 we have to pay for it and throw it after 2-3 years. I see that many others have mentioned this problem.
review_body                        I have had this microwave for just over 3 year...
cooking time                                                                 0.00039
door                                                                          0.9995
customer service & support                                                     4e-05
external size                                                                      0
pre-programmed functions                                                       1e-05
internal size                        

In [304]:
print(review_df.review_body[42])
print(review_df.loc[42])

STAY AWAY FROM PANASONIC MICROWAVES!!! I guess I should be happy, it worked fine for 24 months, which for today's microwaves is probably great. BUT...yesterday it almost burnt down my house!!! Put in 3 frozen microwave pancakes on a microwavable plate, set time for 1:00 minute (I checked to make sure I set timer right), pressed START, and went upstairs for a few minutes. Smelled smoke and ran back down to see my entire first floor engulfed in smoke from fire inside microwave. Unplugged and carefully carried unit outside. Researched this model and I've seen many reports/reviews of arcing and FIRE, just like what happened to me. DANGEROUS!!! If I hadn't come back down, might have set the kitchen and house on fire.
review_body                        STAY AWAY FROM PANASONIC MICROWAVES!!! I guess...
cooking time                                                                 0.00847
door                                                                         0.21134
customer service & supp

In [305]:
print(review_df.review_body[420])
print(review_df.loc[420])

Works as expected.  Nice big size. What else can I say. Was a present for my daughter. Good price. .
review_body                        Works as expected.  Nice big size. What else c...
cooking time                                                                 0.00018
door                                                                          0.0002
customer service & support                                                   0.00129
external size                                                                0.01156
pre-programmed functions                                                     0.94239
internal size                                                                0.00017
power controls                                                               0.00093
short product lifetime                                                       0.00017
product lifetime                                                             0.00019
perceived value                                  

In [306]:
print(review_df.review_body[666])
print(review_df.loc[666])

I wont' go into a ton of detail on this product.  The price was the selling point for me and the good avg reviews.  I will comment on the &#34;sensor reheat&#34; and &#34;sensor cook&#34; functions.  They actually work.  I was actually amazed at that.  After Thanksgiving I put 6 containers in the microwave at once and hit &#34;sensor reheat&#34;.  The containers were:  Turkey, Mashed Potatos, Stuffing, green beans,...hrm, can't remember the last two.  Regardless, of all the containers that came out only the mashed potatos required any further heating.<br /><br />On the sensor cook option it does a nice job overall but it did leave frozen chicken a bit undercooked near the middle of the leg sections.  But yes, I put those sections into it frozen to start with.  I'd count that a win.  (In case you're wondering I was planning on battering/frying that chicken anyhow so a little underdone was perfect.)
review_body                        I wont' go into a ton of detail on this produc...
cook